In [3]:
!pip install langchain cohere wikipedia-api pydantic
!pip install langchain_community

from langchain import PromptTemplate, LLMChain
from langchain.llms import Cohere
from pydantic import BaseModel
from typing import Optional
import wikipediaapi

import getpass
COHERE_API_KEY = "4jZ1rr4LalaGrfHm3TIvImQpc4sD1JUnDF1cl26b"

cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command")

def fetch_ipc_summary():
    user_agent = "IPCChatbot/1.0 (contact: myemail@example.com)"
    wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='en')
    page = wiki_wiki.page("Indian Penal Code")

    if not page.exists():
        raise ValueError("The Indian Penal Code page does not exist on Wikipedia.")

    return page.text[:5000]

ipc_content = fetch_ipc_summary()

class IPCResponse(BaseModel):
    section: Optional[str]
    explanation: Optional[str]

prompt_template = PromptTemplate(
    input_variables=["question"],
    template="""
 You are a legal assistant chatbot specialized in the Indian Penal Code (IPC).
 Refer to the following IPC document content to answer the user's query:

 {ipc_content}

 User Question: {question}

 Provide a detailed answer, mentioning the relevant section if applicable.
 """
)

def get_ipc_response(question: str) -> IPCResponse:
    formatted_prompt = prompt_template.format(ipc_content=ipc_content,
    question=question)

    response = cohere_llm.predict(formatted_prompt)

    if "Section" in response:
        section = response.split('Section')[1].split(':')[0].strip()
        explanation = response.split(':', 1)[-1].strip()
    else:
        section = None
        explanation = response.strip()

    return IPCResponse(section=section, explanation=explanation)

from IPython.display import display
import ipywidgets as widgets

def display_response(response: IPCResponse):
    print(f"Section: {response.section if response.section else 'N/A'}")
    print(f"Explanation: {response.explanation}")

def on_button_click(b):
    user_question = text_box.value
    try:
        response = get_ipc_response(user_question)
        display_response(response)
    except Exception as e:
        print(f"Error: {e}")

text_box = widgets.Text(
value='',
placeholder='Ask about the Indian Penal Code',
description='You:',
disabled=False
)
button = widgets.Button(
description='Ask',
disabled=False,
button_style='',
tooltip='Click to ask a question about IPC',
icon='legal'
)

button.on_click(on_button_click)

display(text_box, button)

<ipython-input-3-4af674442ff6>:13: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command")


Text(value='', description='You:', placeholder='Ask about the Indian Penal Code')

Button(description='Ask', icon='legal', style=ButtonStyle(), tooltip='Click to ask a question about IPC')

<ipython-input-3-4af674442ff6>:49: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = cohere_llm.predict(formatted_prompt)


Section: N/A
Explanation: IPC 301 states "Death caused by child-birth or miscarriage". It states that the following persons are guilty of culpable homicide: 

A person who causes the death of a quick unborn child by an act of negligence; or
A person who causes the death of any child before, during, or after its birth, by an act of negligence; or
A medical practitioner, or any other person, who, unless he owes a duty to the pregnant woman that arises out of a contract or an enactment, delivers a child in good faith and with reasonable care, notwithstanding that the child has sustained an injury fatal to it; or
A person who causes the death of a child, which has been wrongly conceived, or wrongfully delivered, by an act of negligence. 

The penalty for violating this section is a imprisonment of either description for a term which may extend to 10 years, or with fine, or with both.
